In [1]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.layers as tfl
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model

%matplotlib inline

In [2]:
def block(X, kernel, filters, stride, conv_layer = False, training=True):


    X_shortcut = X
    
    # First component 
    X = tfl.Conv2D(filters = filters[0], kernel_size = 1, strides = stride, padding='valid')(X)
    X = tfl.BatchNormalization(axis = 3)(X, training=training)
    X = tfl.Activation('relu')(X)

    
    # Second component 
    X = tfl.Conv2D(filters = filters[1], kernel_size = kernel, strides = 1, padding = 'same')(X)
    X = tfl.BatchNormalization(axis = 3)(X, training = training)
    X = tfl.Activation('relu')(X) 

    # Third component
    X = tfl.Conv2D(filters = filters[2], kernel_size = 1, strides = 1, padding = 'valid')(X)
    X = tfl.BatchNormalization(axis = 3)(X, training = training) 
    
    # SHORTCUT PATH
    if conv_layer:
        X_shortcut = tfl.Conv2D(filters = filters[2], kernel_size = 1, strides = stride, padding = 'valid')(X_shortcut)
        X_shortcut = tfl.BatchNormalization(axis = 3)(X_shortcut, training=training)


    # Final step
    X = tfl.Add()([X, X_shortcut])
    X = tfl.Activation('relu')(X)
    
    return X

In [3]:
def ResNet(Architecture_number, input_shape = (64, 64, 3), classes = 6):

    
    if Architecture_number == 50 :
        identity_layers_stage2 = 2
        identity_layers_stage3 = 3
        identity_layers_stage4 = 5
        identity_layers_stage5 = 2
        
    elif Architecture_number == 101 :
        identity_layers_stage2 = 2
        identity_layers_stage3 = 3
        identity_layers_stage4 = 22
        identity_layers_stage5 = 2
        
    elif Architecture_number == 152 :
        identity_layers_stage2 = 2
        identity_layers_stage3 = 7
        identity_layers_stage4 = 35
        identity_layers_stage5 = 2
        
    else :
        raise Exception("Architecture_number must be one of these numbers: 50 , 101 , 152 ")

    

    X_input = tfl.Input(input_shape)
    
    X = tfl.ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = tfl.Conv2D(filters = 64, kernel_size = 7, strides = 2)(X)
    X = tfl.BatchNormalization(axis = 3)(X)
    X = tfl.Activation('relu')(X)
    X = tfl.MaxPooling2D((3, 3), strides= 2)(X)

    # Stage 2
    X = block(X, kernel = 3, filters = [64, 64, 256], stride = 1, conv_layer = True)
    
    for _ in range(identity_layers_stage2):
        
        X = block(X, kernel = 3, filters = [64, 64, 256], stride = 1)

    
    # Stage 3     
    X = block(X, kernel = 3, filters = [128,128,512], stride = 2, conv_layer = True)
    
    for _ in range(identity_layers_stage3):
        
        X = block(X, kernel =3, filters = [128,128,512], stride = 1)

    
    # Stage 4   
    X = block(X, kernel = 3, filters = [256, 256, 1024], stride = 2, conv_layer = True)
    
    for _ in range(identity_layers_stage4):
        
        X = block(X, kernel = 3, filters = [256, 256, 1024], stride = 1)


    # Stage 5   
    X = block(X, kernel = 3, filters = [512, 512, 2048], stride = 2, conv_layer = True)
    
    for _ in range(identity_layers_stage5):

        X = block(X, kernel = 3, filters = [512, 512, 2048], stride = 1)



    X = tfl.AveragePooling2D((2, 2))(X)

    X = tfl.Flatten()(X)
    
    X = tfl.Dense(classes, activation='softmax')(X)
    
    model = Model(inputs = X_input, outputs = X)

    return model

In [4]:
# define number of layers of ResNet [50 , 101,  152]
Architecture_number = 50
model = ResNet(Architecture_number = Architecture_number, input_shape = (64, 64, 3), classes = 6)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 134, 134, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 64)   9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 64, 64, 64)   256         conv2d[0][0]                     
______________________________________________________________________________________________